In [1]:
### model = DRLPDE_main.maintraining(param='JCPexample5', use_cuda=True)
### calculate error
### Repeat, make a error distribution plot

### Do for different parameters
### eg. number of training steps
###     Size of network?
###     learning rate?
###     Number of Walkers

import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

import DRLPDE_nn
import DRLPDE_functions.DefineDomain
import DRLPDE_functions.EvaluateWalkers

In [ ]:
### Organize parameters related to deep learning solver
############## Walker and Boundary Parameters ############

number_of_runs = 10

problem = 'JCPexample1'

trials = np.zeros((number_of_runs, 2, 5))

# Time step
dt = 1e-4

# exit tolerance
tol = 1e-6

# Number of walkers
num_walkers = 2**13
num_ghost = 256
num_batch = 2**11

# Update walkers
# Options: 
#    move -- moves walkers to one of their new locations
#    remake -- remake walkers at each training step
#    fixed -- keeps walkers fixed
update_walkers = 'move'
update_walkers_every = 1

# Number of boundary points 
num_bdry = 2**10
num_batch_bdry = 2**8

# Number of initial points
num_init = 2**10
num_batch_init = 2**8

############## Training Parameters #######################

# Training epochs
num_epoch = 1000
update_print_every = 1000

# Neural Network Architecture
nn_depth = 20
nn_width = 4

# Weighting of losses
lambda_bell = 1e-2/dt
lambda_bdry = 1
lambda_init = 0

# Learning rate
learning_rate = 1e-2
adam_beta = (0.9,0.999)
weight_decay = 0


In [ ]:
import importlib
    
DRLPDE_param = importlib.import_module("." + problem, package='examples')
    
### Use cuda
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

boundingbox = DRLPDE_param.boundingbox
list_of_dirichlet_boundaries = DRLPDE_param.list_of_dirichlet_boundaries
list_of_periodic_boundaries = DRLPDE_param.list_of_periodic_boundaries
pde_type = DRLPDE_param.pde_type
is_unsteady = DRLPDE_param.is_unsteady
output_dim = DRLPDE_param.output_dim

nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim':DRLPDE_param.x_dim,
            'is_unsteady':DRLPDE_param.is_unsteady ,
            'output_dim':DRLPDE_param.output_dim
            }

move_walkers_param={'x_dim': DRLPDE_param.x_dim,
                    'mu': DRLPDE_param.mu,
                    'dt': dt,
                    'num_batch': num_batch,
                    'num_ghost': num_ghost,
                    'tol': tol
                    }

eval_model_param={'dt': dt,
                    'forcing': DRLPDE_param.forcing}

### Import functions
if is_unsteady:
    # Include time range in bounding box
    boundingbox.append(DRLPDE_param.time_range)
    init_con = DRLPDE_param.init_con
    
    if pde_type == 'NavierStokes':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_NS_unsteady
    elif pde_type == 'Parabolic':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_Parabolic
    elif pde_type == 'StokesFlow':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_Stokes_unsteady
else:
    if pde_type == 'NavierStokes':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_NS_steady
    elif pde_type == 'Elliptic':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_Elliptic
    elif pde_type == 'StokesFlow':
        move_Walkers = DRLPDE_functions.EvaluateWalkers.move_Walkers_Stokes_steady

if pde_type == 'NavierStokes' or 'StokesFlow':
    evaluate_model = DRLPDE_functions.EvaluateWalkers.evaluate_model_NS
else:
    evaluate_model = DRLPDE_functions.EvaluateWalkers.evaluate_model_PDE
    
    move_walkers_param["drift"] = DRLPDE_param.drift
    eval_model_param["reaction"] = DRLPDE_param.reaction


################ Preparing the model #################

#print("Initializing the model")

### Make boundaries defining the domain
Domain = DRLPDE_functions.DefineDomain.Domain(is_unsteady, boundingbox, 
                                                list_of_dirichlet_boundaries,
                                                list_of_periodic_boundaries)

### Initialize the Model
if pde_type == 'NavierStokes' or 'StokesFlow':
    MyNeuralNetwork = DRLPDE_nn.IncompressibleNN
else:
    MyNeuralNetwork = DRLPDE_nn.FeedForwardNN

if DRLPDE_param.loadmodel:
    model = torch.load("savedmodels/" + DRLPDE_param.loadmodel + ".pt")
    print("Using model from savedmodels/" + DRLPDE_param.loadmodel + ".pt")
else:
    model = MyNeuralNetwork(**nn_param).to(dev)

mseloss = nn.MSELoss(reduction = 'mean')
optimizer = optim.Adam(model.parameters(), 
                        lr=DRLPDE_param_solver.learning_rate, 
                        betas=DRLPDE_param_solver.adam_beta, 
                        weight_decay=DRLPDE_param_solver.weight_decay)

### Create Walkers and Boundary points and Organize into DataLoader
RWalkers = DRLPDE_functions.DefineDomain.Walker_Data(num_walkers, boundingbox, Domain.boundaries)
RWalkers_batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

if update_walkers == 'move':
    move_RWalkers = torch.zeros_like(RWalkers.location)

BPoints = DRLPDE_functions.DefineDomain.Boundary_Data(num_bdry, boundingbox, Domain.boundaries, is_unsteady)
BPoints_batch = torch.utils.data.DataLoader(BPoints, batch_size=num_batch_bdry, shuffle=True)

if is_unsteady:
    InitPoints = DRLPDE_functions.DefineDomain.Initial_Data(num_init, boundingbox, Domain.boundaries, init_con)
    InitPoints_batch = torch.utils.data.DataLoader(InitPoints, batch_size=num_batch_init, shuffle=True)

In [3]:
### Analytic Solution

numpts_r = 10
numpts_th =  60

mu = 1
L = 1  # Length of box the circle/sphere sits inside + Radius of Disk/Sphere
v0 = 1

### Analytic Solution
r,th = torch.meshgrid([torch.linspace(0,1, numpts_r), 
                       torch.linspace(0,2*math.pi, numpts_th)])

x1g = r*torch.cos(th)
x2g = r*torch.sin(th)

xg = torch.stack([x1g.reshape(-1), x2g.reshape(-1)], dim=-1).requires_grad_(True)

integral_factor = 2*math.pi*r.detach().cpu().numpy()/(numpts_r-1)/(numpts_th-1)

u1_true = - v0*r*torch.sin(th)
u2_true = v0*r*torch.cos(th)

u_true = torch.stack([u1_true, u2_true], dim=-1)


C:\Users\Kevin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
for step in range(num_epoch):

    # Random Walkers - do in batches
    for Xold, index in RWalkers_batch:

        # Send to GPU and set requires grad flag
        Xold = Xold.to(dev).requires_grad_(True)

        # Evaluate at old location and Move walkers
        Xnew, Uold, outside = move_Walkers(Xold, model, Domain, **move_walkers_param)

        # Evaluate at new location and average
        Unew = evaluate_model(Xold, Xnew, model, **eval_model_param).reshape(num_ghost, 
                                                                                num_batch,
                                                                                output_dim).mean(0)
        
        # Calculate loss
        loss = lambda_bell*mseloss(Uold, Unew.detach())
        loss.backward()

        # If moving walkers save the first ghost walker
        if update_walkers == 'move':
            if any(outside):
                Xnew[:num_batch,:][outside,:] = DRLPDE_functions.DefineDomain.generate_interior_points(torch.sum(outside), 
                                                                                            boundingbox,
                                                                                            Domain.boundaries).to(dev)
            move_RWalkers[index,:] = Xnew[:num_batch].detach().cpu()


    # Boundary Points - do in batches
    for Xbdry, Ubtrue in BPoints_batch:
        Xbdry = Xbdry.to(dev).requires_grad_(True)
        Ubtrue = Ubtrue.to(dev).detach()
        Ubdry = model(Xbdry)
        loss = lambda_bdry*mseloss(Ubdry, Ubtrue)
        loss.backward()

    # Initial Points - do in batches
    if is_unsteady:
        for Xinit, Uinit_true in InitPoints_batch:
            Xinit = Xinit.to(dev).requires_grad_(True)
            Uinit_true = Uinit_true.to(dev).detach()
            Uinit = model(Xinit)
            loss = lambda_bdry*mseloss(Uinit, Uinit_true)
            loss.backward()

    # Make optimization step
    optimizer.step()
    optimizer.zero_grad()

    # Update walkers

    if (step+1) % update_walkers_every == 0:
        if update_walkers == 'move':
            RWalkers.location = move_RWalkers
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)
        elif update_walkers == 'remake':
            RWalkers = DRLPDE_functions.DefineDomain.Walker_Data(num_walkers, boundingbox, Domain.boundaries)
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

In [9]:
error_distribution = np.zeros((number_of_runs, 2))

for run in range(number_of_runs):
    
    

    ug = model(xg)
    ug = ug.reshape([numpts_r, numpts_th, 2])

    u1_approx = ug.select(-1, 0).detach().cpu().numpy()
    u2_approx = ug.select(-1, 1).detach().cpu().numpy()

    x1plot = x1g.detach().cpu().numpy()
    x2plot = x2g.detach().cpu().numpy()

    u1_plot = u1_true.detach().cpu().numpy()
    u2_plot = u2_true.detach().cpu().numpy()

    L2_error = np.sqrt( np.sum( ((u1_approx - u1_plot)**2)*integral_factor) \
                    + np.sum( ((u2_approx - u2_plot)**2 )*integral_factor) )
    Linf_error = np.max( [np.max( np.abs(u1_approx - u1_plot)) , np.max( np.abs(u2_approx - u2_plot))])

    error_distribution[run,0] = L2_error
    error_distribution[run,1] = Linf_error

No errors in first epoch: Training will continue
step = 10/10, 0.172 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.171 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.165 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.171 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.170 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.166 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.165 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.172 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.172 s/step, time-to-go: 0s
No errors in first epoch: Training will continue
step = 10/10, 0.168 s/step, time-to-go: 0s


In [7]:
### Save errors

trials[:,0,1] = error_distribution[:,0]
trials[:,1,1] = error_distribution[:,1]